# Observatory json -> db

In [ ]:
from flask_backend import db
from flask_backend.models import Observatory
import numpy as np

db.create_all()

import json

'''
newObs = Observatory(9999, None, None, None, 10)

db.session.add(newObs)

db.session.commit()

Observatory.query.all()
'''
rep = Observatory.query.filter_by(id=11).first()

#rep.mwid = 2

rep.telescope = 'LIGO/Virgo'

#rep.fullName = 'Neil Gehrels Swift Observatory'

db.session.commit()

#db.session.delete(rep)

Observatory.query.all()


In [ ]:
Observatory.query.all()

In [ ]:
from flask_backend import db
from flask_backend.models import Event
import numpy as np

db.create_all()

import json
eline = Event(99999, 'others',  'others', None, None, None, None, None, None, None, None)


In [ ]:
a = Event.query.filter_by(id=99999).first()

In [ ]:
a

In [ ]:
db.session.delete(a)

In [ ]:
db.session.add(eline)
db.session.commit()

In [ ]:
Observatory.query.all()

# Mission json -> db

In [ ]:
from flask_backend import db
from flask_backend.models_notice import Mission, Notice, Detail, EventNotice
import numpy as np
import json
from IPython.display import clear_output
db.create_all()

In [ ]:
with open('./notice/missions.json') as f:
    for line in f.readlines():
        data = json.loads(line)
        mLine = Mission(data['mid'], data['name'], data['description'], data['basedir']) 
        
        db.session.add(mLine)
        db.session.commit()

In [ ]:
a = Detail.query.filter_by(nid=100)

In [ ]:
a.filter_by(key="NOTICE_DATE").first()

# Notice json -> db

In [ ]:
with open('./notice/notices.json') as f:
    i = 0
    for line in f.readlines():
        data = json.loads(line)
        nLine = Notice(data['nid'], data['mid'], data['file']) 
        
        db.session.add(nLine)
        db.session.commit()

# Detail json -> db

In [ ]:
with open('./notice/details.json') as f:
    i = 0
    for line in f.readlines():
        data = json.loads(line)
        dLine = Detail(i, data['nid'], data['line'], data['key'], data['textval'], data['realval'], str(data['arrval']))
        db.session.add(dLine)
        db.session.commit()
        i+=1
        print(i)
        clear_output(wait=True)

# Parse GRB information

In [ ]:
from flask_backend import db
from flask_backend.models_notice import Notice, Detail, EventNotice
import numpy as np
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
from IPython.display import clear_output
from datetime import datetime
db.create_all()

## Swift

In [ ]:
Swift_filter1 = Detail.query.filter_by(key="COMMENTS").filter_by(textval="This is a GRB.")
Swift_filter2 = Notice.query.filter_by(mid=1)

Swift_nid = []
temp = []
for notice in Swift_filter1:
    temp.append(notice.nid)

for notice in Swift_filter2:
    if notice.id in temp:
        Swift_nid.append(notice.id)
        


In [ ]:
Swift_GRBs = []
i=1
for nid in Swift_nid:
    notice = Detail.query.filter_by(nid=nid)
    trig_num = notice.filter_by(key="TRIGGER_NUM").first().textval[:notice.filter_by(key="TRIGGER_NUM").first().textval.find(",")]
    try:
        ra = float(notice.filter_by(key="GRB_RA").first().realval)
    except:
        continue
    try:
        dec = float(notice.filter_by(key="GRB_DEC").first().realval)
    except:
        continue
        
    c = SkyCoord(ra*u.deg, dec*u.deg)
    ra_hms = "{:.0f}:{:.0f}:{:.2f}".format(*c.ra.hms)
    dec_dms =  "{:.0f}:{:.0f}:{:.2f}".format(c.dec.dms[0], abs(c.dec.dms[1]), abs(c.dec.dms[2]))
    err = notice.filter_by(key="GRB_ERROR").first()
    if notice.filter_by(key="GRB_ERROR").first() != None:
        err = err.realval
    
    try:
        trig_time = Time(float(notice.filter_by(key="GRB_DATE").first().realval) + float(notice.filter_by(key="GRB_TIME").first().realval), format='mjd')
        trigger = trig_time.isot
        
    except:
        print(notice)
    
    Swift_GRBs.append([trig_num, trigger, ra, ra_hms, dec, dec_dms, err])
    print(i, 1, int(nid), int(trig_num), None, None, Time(trigger).datetime, ra, dec, ra_hms, dec_dms, err)    
    enLine = EventNotice(i, 1, int(nid), int(trig_num), None, None, Time(trigger).datetime, ra, dec, ra_hms, dec_dms, err)

    db.session.add(enLine)
    db.session.commit()
    i+=1
    print(i, np.size(Swift_nid))
    clear_output(wait=True)
np.save("Swift", Swift_GRBs)

## Fermi

In [ ]:
Fermi_filter1 = Detail.query.filter_by(key="COMMENTS").filter_by(textval="This is likely a Long GRB.")
Fermi_filter2 = Detail.query.filter_by(key="COMMENTS").filter_by(textval="This is likely a Short GRB.")

Fermi_nid = []
for notice in Fermi_filter1:
    Fermi_nid.append(notice.nid)

for notice in Fermi_filter2:
    Fermi_nid.append(notice.nid)

Fermi_GRBs = []

for nid in Fermi_nid:
    notice = Detail.query.filter_by(nid=nid)
    trig_num = notice.filter_by(key="TRIGGER_NUM").first().textval
    ra = float(notice.filter_by(key="GRB_RA").first().realval)
    dec = float(notice.filter_by(key="GRB_DEC").first().realval)
        
    c = SkyCoord(ra*u.deg, dec*u.deg)
    ra_hms = "{:.0f}:{:.0f}:{:.2f}".format(*c.ra.hms)
    dec_dms =  "{:.0f}:{:.0f}:{:.2f}".format(c.dec.dms[0], abs(c.dec.dms[1]), abs(c.dec.dms[2]))
    err = notice.filter_by(key="GRB_ERROR").first()
    if notice.filter_by(key="GRB_ERROR").first() != None:
        err = err.realval
    
    try:
        trig_time = Time(float(notice.filter_by(key="GRB_DATE").first().realval) + float(notice.filter_by(key="GRB_TIME").first().realval), format='mjd')
        trigger = trig_time.isot
        
    except:
        print(notice)
    
    Fermi_GRBs.append([trig_num, trigger, ra, ra_hms, dec, dec_dms, err])
    print(i, 5, int(nid), int(trig_num), None, None, Time(trigger).datetime, ra, dec, ra_hms, dec_dms, err)    
    enLine = EventNotice(i, 5, int(nid), int(trig_num), None, None, Time(trigger).datetime, ra, dec, ra_hms, dec_dms, err)

    db.session.add(enLine)
    db.session.commit()
    i+=1
    print(i, np.size(Fermi_nid))
    clear_output(wait=True)
np.save("Fermi", Fermi_GRBs)

# Association

In [1]:
from flask_backend.models import Circular, Event, Observatory, MMMW,  Detail, EventNotice

from flask_backend import db
import numpy as np
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u
from IPython.display import clear_output
from datetime import datetime

db.create_all()

In [ ]:
swift_n = np.load("./etc/BAT_names.npy")

fermi_n = np.load("./etc/GBM_names_fixed.npy")


GRB_list = swift_n[:,1].tolist() + fermi_n[:,1].tolist()

GRB_list = sorted(list(set(GRB_list)))

evt_list = []
i = 1
for grb in GRB_list:
    try:
        s_idx = swift_n[:,1].tolist().index(grb)
        s_id = swift_n[s_idx][0]
    except:
        s_id = None
        pass

    try:
        f_idx = fermi_n[:,1].tolist().index(grb)
        f_id = fermi_n[f_idx][0][3:]
    except:
        f_id = None
        pass
    evt_list.append([i, grb, s_id, f_id])
    i+=1
evt_list = np.asarray(evt_list)

## Swift

In [ ]:
events = EventNotice.query.all()
for evt in events:
    if str(evt.tid) in evt_list[:,2]:
        evt_n = EventNotice.query.filter_by(tid=evt.tid).first()
        evt_idx = evt_list[:,2].tolist().index(str(evt.tid))
        print(evt_list[evt_idx][1], evt_n.trigger)
        evt_n.evtid = evt_list[evt_idx][0]
        evt_n.event = evt_list[evt_idx][1]
        db.session.commit()

## Fermi

In [ ]:
events = EventNotice.query.all()
for evt in events:
    foundF = False
    trig_time = Time(evt.trigger)
    name1 = trig_time.isot[:10].replace("-", "")[2:]
    name2 = "{:.3f}".format((trig_time.datetime.hour*3600+trig_time.datetime.minute*60+trig_time.datetime.second)/(24*60*60))
    trig_DT = str(name1)+str(name2)
    trig_DT = trig_DT.replace("0.", "")
    if trig_DT in evt_list[:,3]:
        evt_idx = evt_list[:,3].tolist().index(str(int(trig_DT)))
        print(trig_DT, evt_list[evt_idx])
        foundF = True
    elif int(trig_DT) in (evt_list[:,3][evt_list[:,3]!=None].astype("int")+1):
        evt_idx = evt_list[:,3].tolist().index(str(int(trig_DT)-1))
        print(trig_DT, evt_list[evt_idx])
        foundF = True
    elif int(trig_DT) in (evt_list[:,3][evt_list[:,3]!=None].astype("int")-1):
        evt_idx = evt_list[:,3].tolist().index(str(int(trig_DT)+1))
        print(trig_DT, evt_list[evt_idx])
        foundF = True
    elif evt.mid == 5 and sum(fermi_n[:,1] == 'GRB'+trig_DT[:6]+'A') > 0:
        print(trig_DT, "Not")
    if foundF:
        evt_n = EventNotice.query.filter_by(tid=evt.tid).first()
        evt_n.evtid = evt_list[evt_idx][0]
        evt_n.event = evt_list[evt_idx][1]
        db.session.commit()

In [ ]:
for evt in evt_list:
    eNotice = EventNotice.query.filter_by(evtid = evt[0]).all()
    if len(eNotice) == 1:
        eNotice = eNotice[0]
        Eline = Event(eNotice.evtid, eNotice.event, 'GRB', eNotice.trigger, eNotice.ra, eNotice.dec, eNotice.ra_hms, eNotice.dec_dms, eNotice.error)
    elif len(eNotice) == 2:
        eNotice_s = EventNotice.query.filter_by(evtid = evt[0]).filter_by(mid=1).first()
        eNotice_f = EventNotice.query.filter_by(evtid = evt[0]).filter_by(mid=5).first()
        print(eNotice_s.trigger, eNotice_f.trigger, Time(eNotice_s.trigger)- Time(eNotice_f.trigger))
        if (Time(eNotice_s.trigger)- Time(eNotice_f.trigger)) > 0:
            trigger = eNotice_f.trigger
        else:
            trigger = eNotice_s.trigger
        Eline = Event(eNotice_s.evtid, eNotice_s.event, 'GRB', trigger, eNotice_s.ra, eNotice_s.dec, eNotice_s.ra_hms, eNotice_s.dec_dms, eNotice_s.error)
    else:
        Eline = Event(evt[0], evt[1], 'GRB', None, None, None, None, None, None)
    db.session.add(Eline)
    db.session.commit()
    

In [ ]:
evts = Event.query.all()
for evt in evts:
    if evt.ra != None:
        c = SkyCoord(evt.ra*u.deg, evt.dec*u.deg)
        ra_hms = "{:.0f}:{:.0f}:{:.2f}".format(*c.ra.hms)
        dec_dms =  "{:.0f}:{:.0f}:{:.2f}".format(c.dec.dms[0], abs(c.dec.dms[1]), abs(c.dec.dms[2]))
        evt.ra_hms = ra_hms
        evt.dec_dms = dec_dms
        db.session.commit()

In [ ]:
evts = Event.query.all()

evtList = []
for evt in evts:
    if len(evt.notices) == 0:
        evtList.append([evt.id, evt.event, 0])
    else:
        for en in  evt.notices:
            evtList.append([evt.id, evt.event, en.tid])

evtList=np.asarray(evtList)

In [5]:
subject = Circular.query.filter_by(id=27347).first().subject

In [7]:
ListOfObs = Observatory.query.all()

title = " "+subject[subject.find(":")+1:]+" "

title = title.replace("Fermi GRB", "GRB") # This is due to MASTER-NET reports
title = title.replace(":", " ")
title = title.replace("_", " ")

ListOfTel = []

for obs in ListOfObs:
    if obs.id < 1000:
        ListOfTel.append([obs.id, obs.telescope, obs.detector, obs.fullName])

ListOfTel = np.asarray(ListOfTel)

foundTel = [tel[0] for tel in ListOfTel if (title.find(" "+tel[1]+" ")>=0) or (title.find(" "+tel[1]+"/")>=0) or (title.find(" "+tel[1]+"-")>=0) or (title.find("("+tel[1]+")")>=0) or (title.find(" "+tel[1].upper()+" ")>=0)]
foundTel += [tel[0] for tel in ListOfTel if (tel[0] not in foundTel) and ((tel[2]!=None) and (((title.find(" "+tel[2]+" ")>=0) or (title.find("("+tel[3]+")")>=0) or (title.find(" "+tel[3].upper()+" ")>=0))))]
foundTel += [tel[0] for tel in ListOfTel if (tel[0] not in foundTel) and (((title.find(" "+tel[3]+" ")>=0) or (title.find(" "+tel[3]+"/")>=0) or (title.find(" "+tel[3]+"-")>=0) or (title.find("("+tel[3]+")")>=0) or (title.find(" "+tel[3].upper()+" ")>=0)))]
foundTel += [tel[0] for tel in ListOfTel if (tel[0] not in foundTel) and (np.size(foundTel) == 0) and ((tel[1] == 'LIGO/Virgo' or tel[1] == 'IceCube' or tel[1] == 'ANTARES') and (subject.find(tel[1])>=0))]

if np.size(foundTel)== 1:
    foundObs = ListOfTel[ListOfTel[:,0]==foundTel[0]]
    print(foundObs[0][0])


901
